In [16]:
from sentence_transformers import SentenceTransformer
import dotenv
import json
import os
import sys
from pathlib import Path
from app.logging.logger import logger

from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct 
import fitz  # PyMuPDF for reading PDF files
import typer
import uuid
from openai import OpenAI
from dotenv import load_dotenv
import json
load_dotenv()


from app.config.path_config import RAW_DATA_DIR



/Users/saraevsviatoslav/Documents/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2025-06-24 11:34:33] INFO - PROJ_ROOT path is: /Users/saraevsviatoslav/Documents/ai_knowledge_assistant


In [3]:
app = typer.Typer()

raw_path = Path(RAW_DATA_DIR)

# Get required Qdrant credentials from environment
qdrant_key = os.getenv('QDRANT_API_KEY')
qdrant_url = os.getenv('QDRANT_HOST')

# Validate that required environment variables exist
env_var = ['QDRANT_API_KEY', 'QDRANT_HOST']
missing_env = [var for var in env_var if os.getenv(var) is None]
if missing_env:
    logger.error(f"Missing environment variables: {', '.join(missing_env)}")
    raise SystemExit(1)

# Initialize Qdrant client
qdrant_client = QdrantClient(
    url=qdrant_url,
    api_key=qdrant_key
)



/var/folders/gc/0vl3v58x5t7d7bz8vwtmtbt00000gn/T/ipykernel_45158/1381222653.py:17: UserWarning: Failed to obtain server version. Unable to check client-server compatibility. Set check_compatibility=False to skip version check.
  qdrant_client = QdrantClient(


In [4]:
def extract_text_from_pdf(path: Path) -> str | None:
    """
    Extract text content from a PDF file.
    Returns None if an error occurs.
    """
    try:
        with fitz.open(path) as doc:
            return "\n".join(page.get_text() for page in doc)
    except Exception as e:
        logger.error(f"Error opening {path}: {e}")
        return None
    
def load_question_from_json(json_path: Path) -> list[str]:
    """
    Load a list of question strings from a JSON file.
    Assumes each item in JSON is a dict with a 'text' field.
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return [entry['text'] for entry in data if 'text' in entry]

def load_all_documents(folder: Path) -> list[str]:
    """
    Load text content from all supported documents (PDF and JSON).
    Returns a list of text strings ready for embedding.
    """
    texts = []

    # Load text from PDF files
    for pdf in folder.glob("*.pdf"):
        text = extract_text_from_pdf(pdf)
        if text and text.strip():
            texts.append(text)

    # Load questions from JSON files
    for js in folder.glob("*.json"):
        try:
            questions = load_question_from_json(js)
            texts.extend(questions)
        except Exception as e:
            logger.warning(f"Could not load {js.name}: {e}")

    return texts



In [5]:
def prepare_points(titles: list[str], documents: list[str], embeddings) -> list[PointStruct]:
    """
    Prepare PointStruct objects for uploading to Qdrant.
    Also (re)creates the 'docs' collection if it does not exist.
    """
    if not qdrant_client.collection_exists('arXiv'):
        qdrant_client.recreate_collection(
            collection_name='arXiv',
            vectors_config=VectorParams(size=embeddings.shape[1], distance=Distance.COSINE)
        )

    points = [
        PointStruct(
            id=uuid.uuid4().int >> 64,
            vector=embeddings.data[0].embedding.tolist(),
            payload={"text": doc, "title": title},
        )
        for title, doc, vector in zip(titles, documents, embeddings)
    ]
    return points



In [17]:

openai_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(
    api_key=openai_key
)

# Load embedding model
#model = SentenceTransformer('BAAI/bge-base-en-v1.5')

# Load all documents (PDF + JSON)
#documents = load_all_documents(raw_path)
path = '/Users/saraevsviatoslav/Documents/ai_knowledge_assistant/data/external/arxiv_filtered.json'
data = []
with open(path, 'r', encoding='utf-8') as f:
    data = json.load(f)





In [1]:
print(results)

NameError: name 'results' is not defined

In [18]:
from tqdm import tqdm

embeddings = []
contents = []
titles = []

for entry in tqdm(data, desc="Embedding progress"):
    abstract = entry['abstract']
    title = entry['title']
    
    # Generate embedding
    embedding = client.embeddings.create(
        input=abstract,
        model='text-embedding-3-small',
    )
    
    embeddings.append(embedding)
    contents.append(abstract)
    titles.append(title)

Embedding progress:   0%|          | 0/89423 [00:00<?, ?it/s]

Embedding progress:   1%|▏         | 1229/89423 [06:04<7:16:22,  3.37it/s] 


KeyboardInterrupt: 

In [ ]:
import asyncio
from openai import AsyncOpenAI
from tqdm.asyncio import tqdm_asyncio

client = AsyncOpenAI(api_key=...)

async def get_embedding(entry):
    abstract = entry['abstract']
    title = entry['title']
    try:
        response = await client.embeddings.create(
            input=abstract,
            model='text-embedding-3-small',
        )
        return {
            "embedding": response.data[0].embedding,
            "title": title,
            "abstract": abstract,
        }
    except Exception as e:
        print(f"Error: {e}")
        return None

async def main(data):
    results = []
    for i in tqdm_asyncio.tqdm_asyncio.as_completed([get_embedding(e) for e in data]):
        r = await i
        if r: results.append(r)
    return results

# запуск
final_results = asyncio.run(main(data[:10000]))  # начинай с части, например 10 тыс

In [13]:
import json
import re

def filter_jsons(path_in: str, path_out: str):
    keywords = ['rag', 'graph', 'llm', 'retrieval', 'gnn', 'gan']
    pattern = re.compile(r'\b(?:' + '|'.join(re.escape(k) for k in keywords) + r')\b', re.IGNORECASE)
    
    filtered = []
    with open(path_in, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                item = json.loads(line)
                text = item.get('abstract', '') + ' ' + item.get('title', '')
                if pattern.search(text) and item.get('categories', '').startswith('cs.'):
                    filtered.append(item)
            except json.JSONDecodeError:
                continue  # Пропускаем кривые строки
    
    with open(path_out, 'w', encoding='utf-8') as f:
        json.dump(filtered, f, indent=2, ensure_ascii=False)

In [14]:
path_in = '/Users/saraevsviatoslav/Documents/ai_knowledge_assistant/data/external/arxiv-metadata-oai-snapshot.json'
path_out = '/Users/saraevsviatoslav/Documents/ai_knowledge_assistant/data/external/arxiv_filtered.json'
filter_jsons(path_in, path_out)

In [ ]:
# Prepare and upload points to Qdrant
points = prepare_points(titles, contents, embeddings)
qdrant_client.upload_points(collection_name='arXiv', points=points)

logger.success(f"Uploaded {len(points)} documents to Qdrant successfully.")


In [21]:
import asyncio
import aiohttp
import json
from tqdm.asyncio import tqdm
from typing import List
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
assert api_key, "OPENAI_API_KEY is not set"

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Путь к входному и выходному файлам
input_path = "/Users/saraevsviatoslav/Documents/ai_knowledge_assistant/data/external/arxiv_filtered.json"
output_path = "/Users/saraevsviatoslav/Documents/ai_knowledge_assistant/data/external/new.jsonl"

# Ограничение по параллельности (не превышай лимит OpenAI)
MAX_CONCURRENT_REQUESTS = 5


async def fetch_embedding(session, abstract: str, title: str):
    url = "https://api.openai.com/v1/embeddings"
    payload = {
        "input": abstract,
        "model": "text-embedding-3-small"
    }

    async with session.post(url, headers=headers, json=payload) as response:
        if response.status == 200:
            result = await response.json()
            return {
                "title": title,
                "abstract": abstract,
                "embedding": result["data"][0]["embedding"]
            }
        else:
            error = await response.text()
            print(f"Error {response.status}: {error}")
            return None


async def process_entries(entries: List[dict]):
    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
    connector = aiohttp.TCPConnector(limit=MAX_CONCURRENT_REQUESTS)

    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = []

        async def sem_task(entry):
            async with semaphore:
                return await fetch_embedding(session, entry["abstract"], entry["title"])

        for entry in entries:
            tasks.append(sem_task(entry))

        results = []
        for f in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Embedding progress"):
            result = await f
            if result:
                results.append(result)
                with open(output_path, "a", encoding="utf-8") as f_out:
                    f_out.write(json.dumps(result) + "\n")
        return results


def load_data(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)



entries = load_data(input_path)
asyncio.run(process_entries(entries))

In [ ]:
entries = load_data(input_path)
asyncio.run(process_entries(entries))

/opt/homebrew/Cellar/python@3.11/3.11.11/Frameworks/Python.framework/Versions/3.11/lib/python3.11/json/decoder.py:353: RuntimeWarning: coroutine 'process_entries' was never awaited
  obj, end = self.scan_once(s, idx)


RuntimeError: asyncio.run() cannot be called from a running event loop